In [ ]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import pandas as pd
from urllib.request import urlopen
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
import time
import json
import re
import csv

In [ ]:
%matplotlib inline # Displays plots inside notebook

In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.walgreens.com/storelistings/storesbystate.jsp?requestType=locator")     

In [ ]:
def check_exists_by_xpath(xpath):  # Function to check if an element exists on webpage
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

In [ ]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "DC", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY",
          "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH",
          "OK", "OR", "PA", "PR", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "VI", "WA", "WV", "WI", "WY"]

AL_CA = states[0:5]
CO_HI = states[5:12]
ID_KY = states[12:18]
LA_MT = states[18:27]
NE_NY = states[27:33]
NC_TN = states[33:44]
TX_WY = states[44:53]  # Remove PR & VI during cleaning

print(AL_CA)
print(CO_HI)
print(ID_KY)
print(LA_MT)
print(NE_NY)
print(NC_TN)
print(TX_WY)

In [ ]:
addresses_dict = {'address': [], 'state': [], 'zip': []}

for state in TX_WY:
    
    driver.get("https://www.walgreens.com/storelistings/storesbycity.jsp?requestType=locator&state=" + state)
    
    state_stores = driver.find_elements_by_xpath('//ul[@class="col-xl-4 col-lg-4 col-md-4"]')
    city_list = "\n".join([x.text for x in state_stores]).split("\n")
    cities = []
    
    for city in city_list:
        cities.append(city.split(",")[0].replace(" ", "%20"))
    
    base_url = "https://www.walgreens.com/storelocator/find.jsp?requestType=locator&state="+ state + "&city=%s&from=localSearch"
    url_list = []
    
    for city in cities:
        url_list.append(base_url % (city))
    
    for url in url_list:
        driver.get(url)
    
        if check_exists_by_xpath('//span[@class="btn btn__tint-blue"]'):
            load_button = driver.find_element_by_xpath('//span[@class="btn btn__tint-blue"]')
            while load_button:
                try:
                    load_button.click()
                    time.sleep(2)
                    load_button = driver.find_element_by_xpath('//span[@class="btn btn__tint-blue"]')
                except NoSuchElementException:
                    break

        addresses = driver.find_elements_by_xpath('//div[@class="address"]')

        for address in addresses:

            addresses_dict['address'].append(address.text.replace("\n", ", "))
            addresses_dict['state'].append(re.search(",\s([A-Z]{2})", address.text).group().replace(", ", ""))
            addresses_dict['zip'].append(re.search("\d{5}(?:[-\s]\d{4})?$", address.text).group())

        time.sleep(1)

TX_WY_df = pd.DataFrame(addresses_dict)

In [ ]:
# CLEAN MAKING SURE TO START WITH NUMBER - DELETE IF NOT PRESENT
# DFNAME.drop_duplicates().to_csv('LA_MT.csv', index=False)

AL_CA_df = AL_CA_df.drop_duplicates()
CO_HI_df = CO_HI_df.drop_duplicates()
ID_KY_df = ID_KY_df.drop_duplicates()
LA_MT_df = LA_MT_df.drop_duplicates()
NE_NY_df = NE_NY_df.drop_duplicates()
NC_TN_df = NC_TN_df.drop_duplicates()
TX_WY_df = TX_WY_df.drop_duplicates()

## Creating final dataframe

dataframes = [AL_CA_df, CO_HI_df, ID_KY_df, LA_MT_df, NE_NY_df, NC_TN_df, TX_WY_df]
finaldf = pd.concat(dataframes)
finaldf.drop_duplicates().to_csv('finaldf.csv', index=False)

In [ ]:
finaldf['zip'] = finaldf['zip'].astype(str) ## CHANGE ZIP COLUMN TO STRING
finaldf = finaldf.loc[finaldf['zip'].str.len() > 3]  ## FILTER OUT PR and international strings

for index, row in finaldf.iterrows(): ## Format zip codes to be len 5
    if len(row['zip']) == 4:
        row['zip'] = '0' + row['zip']

finaldf.loc[finaldf['zip'].str.len() == 4] ## Should return empty DF if correctly executed

for index, row in finaldf.iterrows(): ## Removes text prior to an address listing (e.g. Walgreens Pharmacy, <address>)
    if row['address'][0].isalpha():
        row['address'] = row['address'].partition(",")[2].strip()

In [ ]:
zcdf = pd.read_csv('zipcounty.csv')
zcdf['fips'] = zcdf['fips'].astype(str) ## CHANGE ZIP COLUMN TO STRING
zcdf.loc[zcdf['fips'].str.len() == 4]  ## FILTER OUT PR and international strings

for index, row in zcdf.iterrows(): ## Format zip codes to be len 5
    if len(row['fips']) == 4:
        row['fips'] = '0' + row['fips']

finaldf.loc[finaldf['zip'].str.len() == 4] ## Should return empty DF if correctly executed

In [ ]:
def get_county(response):  # Filter County from Google API response

    for i in range(len(response['results'][0]['address_components'])):
        if "County" in response['results'][0]['address_components'][i]['long_name']:
            return response['results'][0]['address_components'][i]['long_name']
        elif "Parish" in response['results'][0]['address_components'][i]['long_name']:
            return response['results'][0]['address_components'][i]['long_name']

In [ ]:
# Initialize API parameters

API_KEY = ''
base_url = 'https://maps.googleapis.com/maps/api/geocode/json?' 
addresses = finaldf['address']
counties = {'county': []}

In [ ]:
for address in addresses:
    parameters = {
                'key': API_KEY,
                'address': address
                }
    
    response = requests.get(base_url, parameters).json()
    counties['county'].append(get_county(response))  # Extract county from request

countydf = pd.DataFrame(counties)
countydf.to_csv('counties.csv', index=False)

In [ ]:
countydf = pd.read_csv('counties.csv')
countydf.isna().sum() ## Initially 144 na values -> required manual cleaning, validated for 0 NaN results
finaldf = pd.concat([finaldf, countydf], axis=1)  ## adding county column to data

In [ ]:
lat = response['results'][0]['geometry']['location']['lat']  # May decide to request lat,long data depending on project needs
lng = response['results'][0]['geometry']['location']['lng']
lat, lng

In [ ]:
# Check first using zip code then str.lower of county and state
# Can check fips using sort(low->high) and checking similar state fips (ignore county fips)

In [ ]:
# https://data.world/niccolley/us-zipcode-to-county-state  ## prepping zipcode-fips file prior to merge
zcdf = pd.read_csv('zipcounty.csv')
zcdf = zcdf.astype({'zip':'str', 'fips':'str'})
for index, row in zcdf.iterrows(): ## Format zip codes to be len 5
    if len(row['fips']) == 4:
        row['fips'] = '0' + row['fips']

completedf = finaldf.merge(zcdf, on=['zip', 'county'], how='left').drop(columns=['city', 'state_y']).rename({'state_x': 'state'}, axis=1)

In [ ]:
# Cleaning missing FIPS data
missingfipsdf = completedf[completedf.isnull().any(axis=1)].drop(columns=['fips', 'class_fips']) ## manually cleaning missing fips
missingfipsdf.to_csv('missingfips.csv', index=False)
completedf.to_csv('completed.csv', index=False)

In [ ]:
completeddf = pd.read_csv('completed.csv', dtype = str)
allfips = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv", dtype={"fips": str})
allfips['unemp'] = allfips['unemp'].apply(lambda x: 0)  # make 0 for all counties listed
allfips = allfips.rename(columns={"unemp":"count"})     # rename columns

fipscountdf = pd.DataFrame(completedf['fips'].value_counts(dropna=False)).reset_index()
fipscountdf = fipscountdf.rename(columns={"index":"fips","fips": "count"})  # rename columns

completefips = pd.concat([fipscountdf, allfips]).groupby(['fips']).sum().reset_index()  # adding counts of my fips with external fips dataset

In [ ]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [ ]:
## Chloropleth map for county-level Walgreens location information

fig = px.choropleth(completefips, geojson=counties, locations='fips', color='count',
                           color_continuous_scale="Viridis",
                           range_color=(0, 10),
                           scope="usa",
                           labels={'count':'count'}
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth(completefips, geojson=counties, locations='fips', color='count',
                           color_continuous_scale="plasma",
                           range_color=(0, 10),
                           scope="usa",
                           labels={'count':'count'}
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
#https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-total.html#par_textimage

In [ ]:
countypopdf = pd.read_csv("countypop.csv")  # crate county population DataFrame

In [ ]:
allcountydf = completedf.merge(countypopdf, on=['state', 'county'], how='left')
allcountydf.to_csv('final.csv', index=False)  # export for manual clearning
allcountydf.isnull().sum()  ## Count number of null 
allcountydf[allcountydf.isnull().any(axis=1)] ## See rows for manual data cleaning

In [ ]:
allcountydf = pd.read_csv('final.csv')
allcountydf[allcountydf.address.duplicated()] ## validation to check for duplicate addresses created due to error

In [ ]:
## Loading NYT covid data (github.com/nytimes/covid-19-data)
# https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv

In [ ]:
covidcountydf = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
covidcountydf = covidcountydf.loc[covidcountydf.date == '2021-02-05'].reset_index(drop = True)
covidcountydf[covidcountydf.isnull().any(axis=1)]
covidcountydf = covidcountydf[covidcountydf['fips'].notna()]

In [ ]:
covidcountydf = covidcountydf.astype({'fips': 'int64'})
covidcountydf = covidcountydf.astype({'fips': 'str'})

In [ ]:
allcovidcountydf = allcountydf.merge(covidcountydf[{'fips','cases','deaths'}], on=['fips'], how='left')

In [ ]:
covidcountycount = pd.read_csv('covidcountycount.csv')

In [ ]:
allcovidcountydf[allcovidcountydf.isnull().any(axis=1)].fips.unique()

In [ ]:
covidtotalfips = completefips.merge(covidcountydf[{'fips','cases','deaths'}], on=['fips'], how='left')
fig = px.choropleth(covidcountydf, geojson=counties, locations='fips', color='cases',
                           color_continuous_scale="Viridis",
                           range_color=(0, 10),
                           scope="usa",
                           labels={'count':'count'}
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
covidcountycount = completefips.merge(covidcountydf, on=['fips'], how='left')
covidcountycount = covidcountycount[covidcountycount.cases.notna()]

In [ ]:
completefips.fips = completefips.fips.apply(lambda x: x[1:] if x[0] == '0' else x)
covidcountycount = covidcountycount.astype({'cases': 'int64', 'fips':'str'})
covidcountycount.fips = covidcountycount.fips.apply(lambda x: '0' + x if len(x) == 4 else x)
covidcountycount['store_ratio'] = 100*covidcountycount['count']/covidcountycount['cases']

covidcountycount = covidcountycount.rename(columns = {'count':'store_count'})
covidcountycount = covidcountycount[['fips', 'county', 'state', 'cases', 'deaths', 'store_count', 'store_ratio']]
covidcountycount = covidcountycount.sort_values(by=['store_ratio'], ascending=False)

In [ ]:
countyfinal = covidcountycount.copy()  # Preparing final county-level dataframe

## BUREAU OF ECONOMIC ANALYSIS
## CAINC1 Personal Income Summary: Personal Income, Population, Per Capita Personal Income

countyincome = pd.read_csv('countyincome.csv') 
countyincome = countyincome.astype({'LineCode':'int64'})
countyincome = countyincome.astype({'fips':'str'})
countyincome.fips = countyincome.fips.apply(lambda x: '0' + x if len(x) == 4 else x)

In [ ]:
totalincome = countyincome[(countyincome['LineCode'] == 1)] ## Extracting DFs from countyincome
countypopulation = countyincome[(countyincome['LineCode'] == 2)]
incomepercapita = countyincome[(countyincome['LineCode'] == 3)] ## Incomepercapita = rounded(totalincome/countypopulation)

In [ ]:
totalincome
totalincome.sort_values(by=['fips'])
countyfinal = countyfinal.merge(totalincome, on=['fips'], how='left').drop(columns={'GeoName', 'LineCode', 'Description'}).rename(columns={'2019':'totalincome_thousands'})
countyfinal = countyfinal.merge(countypopulation, on=['fips'], how='left').drop(columns={'GeoName', 'LineCode', 'Description'}).rename(columns={'2019':'population'})
countyfinal = countyfinal.merge(incomepercapita, on=['fips'], how='left').drop(columns={'GeoName', 'LineCode', 'Description'}).rename(columns={'2019':'incomepercapita'})
countyfinal = countyfinal[~countyfinal.state.str.contains("Puerto")] # Filtering out Puerto Rico 

In [ ]:
countyfinal.to_csv('countyfinal.csv', index=False)
countyfinal.loc[countyfinal.population.isnull()] ## Manually cleaning missing values (ALL VA)

## NOTE: A MAJORITY IN VA ARE COMBINED -> IGNORE IN MAP

countyfinal.describe()

In [ ]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

fig = px.choropleth(countyfinal, geojson=counties, locations='fips', color='cases',
                           color_continuous_scale="portland",
                           range_color=(0, 50000),
                           scope="usa",
                           labels={'count':'count'}
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth(countyfinal, geojson=counties, locations='fips', color='store_count',
                           color_continuous_scale="portland",
                           range_color=(0, 12),
                           scope="usa",
                           labels={'count':'count'}
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# COVID CASES BY COUNTY

fig = px.choropleth(countyfinal, geojson=counties, locations='fips', color='cases',
                           color_continuous_scale="viridis",
                           range_color=(0, 20000),
                           scope="usa",
                           labels={'count':'count'}
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# TOTAL INCOME MAP

fig = px.choropleth(countyfinal, geojson=counties, locations='fips', color='totalincome_thousands',
                           color_continuous_scale="plasma",
                           range_color=(0, 1e7),
                           scope="usa",
                           labels={'count':'count'}
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# TOTAL Population MAP

fig = px.choropleth(countyfinal, geojson=counties, locations='fips', color='population',
                           color_continuous_scale="rdbu_r",
                           range_color=(0, 2e5),
                           scope="usa",
                           labels={'count':'count'}
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# TOTAL INCOME/CAPITA

fig = px.choropleth(countyfinal, geojson=counties, locations='fips', color='incomepercapita',
                           color_continuous_scale="plasma",
                           range_color=(0, 8e4),
                           scope="usa",
                           labels={'count':'count'}
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()